## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# cutoff_value = 200
cutoff_value = 10
application_types_to_replace = list(application_type_counts[application_type_counts < cutoff_value].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# cutoff_value = 1000
cutoff_value = 30
classifications_to_replace = list(classification_counts[classification_counts< cutoff_value].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


In [ ]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
application_df_encoded = pd.get_dummies(application_df, columns=categorical_columns)

# Split our preprocessed data into our features and target arrays
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Feature Importance using the Random Forest Model

In [ ]:
# Train a simple model to get feature importance
from sklearn.ensemble import RandomForestClassifier

# Instantiate a RandomForestClassifier (or any other model)
model = RandomForestClassifier()

# Fit the model on your data
model.fit(X_train_scaled, y_train)

# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame with feature names and their importance scores
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the DataFrame by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Print or visualize the feature importances
print(feature_importance_df)

# Select a threshold for importance and keep only important features
selected_features = list(feature_importance_df[feature_importance_df['Importance'] > 0.001]['Feature'])
# Subset your data based on selected features
feature_indices = [X.columns.get_loc(feature) for feature in selected_features]

# Subset your data based on selected features
X_train_selected = X_train_scaled[:, feature_indices]
X_test_selected = X_test_scaled[:, feature_indices]

                         Feature  Importance
0                        ASK_AMT    0.392760
14  AFFILIATION_CompanySponsored    0.136980
16       AFFILIATION_Independent    0.114666
9            APPLICATION_TYPE_T5    0.028972
46      ORGANIZATION_Association    0.023878
..                           ...         ...
22          CLASSIFICATION_C1230    0.000126
18             AFFILIATION_Other    0.000072
50                      STATUS_0    0.000060
51                      STATUS_1    0.000058
43                USE_CASE_Other    0.000053

[63 rows x 2 columns]


In [ ]:
X_train_selected

array([[-0.01334191, -0.92331648,  0.92918396, ..., -0.07317296,
        -0.06523281, -0.04278355],
       [-0.03089076,  1.08305226, -1.07621315, ..., -0.07317296,
        -0.06523281, -0.04278355],
       [-0.03089076, -0.92331648,  0.92918396, ..., -0.07317296,
        -0.06523281, -0.04278355],
       ...,
       [-0.03089076, -0.92331648,  0.92918396, ..., -0.07317296,
        -0.06523281, -0.04278355],
       [-0.03089076,  1.08305226, -1.07621315, ..., -0.07317296,
        -0.06523281, -0.04278355],
       [-0.02964949, -0.92331648,  0.92918396, ..., -0.07317296,
        -0.06523281, -0.04278355]])

## Optimize the Model

### Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_selected.shape[1]
# input_features = X_train.shape[1]
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=80, input_dim=input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu"))

# Fourth hidden layer
# nn.add(tf.keras.layers.Dense(units=65, activation="sigmoid"))
# Add the remaining hidden layers
# for i in range(3):
#     nn.add(
#         tf.keras.layers.Dense(
#             units=1,
#             activation='sigmoid'
#         )
#     )
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3360      
                                                                 
 dense_1 (Dense)             (None, 50)                4050      
                                                                 
 dense_2 (Dense)             (None, 20)                1020      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 8451 (33.01 KB)
Trainable params: 8451 (33.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# nn.compile(loss='binary_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [ ]:
# Train the model
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# fit_model = nn.fit(X_train_selected, y_train, epochs=100, batch_size=64, validation_data=(X_test_selected, y_test))
fit_model = nn.fit(X_train_selected, y_train, validation_data=(X_test_selected, y_test), epochs=200, callbacks=[early_stopping])
# fit_model = nn.fit(X_train_selected, y_train, epochs=200, batch_size=64)
# fit_model = nn.fit(X_train_scaled, y_train, epochs=100)
#fit_model = nn.fit(X_train_scaled, y_train, epochs=120, batch_size=32)

Epoch 1/200
804/804 [==============================] - 8s 6ms/step - loss: 0.5735 - accuracy: 0.7181 - val_loss: 0.5614 - val_accuracy: 0.7234
Epoch 2/200
804/804 [==============================] - 6s 8ms/step - loss: 0.5540 - accuracy: 0.7288 - val_loss: 0.5585 - val_accuracy: 0.7270
Epoch 3/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5511 - accuracy: 0.7285 - val_loss: 0.5535 - val_accuracy: 0.7264
Epoch 4/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5491 - accuracy: 0.7312 - val_loss: 0.5533 - val_accuracy: 0.7284
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5473 - accuracy: 0.7319 - val_loss: 0.5544 - val_accuracy: 0.7273
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5463 - accuracy: 0.7331 - val_loss: 0.5515 - val_accuracy: 0.7264
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5452 - accuracy: 0.7327 - val_loss: 0.5555 - val_accuracy: 0.7234

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_selected,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5487 - accuracy: 0.7310 - 390ms/epoch - 1ms/step
Loss: 0.5487109422683716, Accuracy: 0.7309620976448059


In [ ]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
